# 💡 Train an UAV Detector with MMDetection and Weights and Biases


In [1]:
import os
import os.path as osp
import torch
import torchvision
import numpy as np

# MMDetection
import mmdet
print(mmdet.__version__)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import set_random_seed

# MMCV
import mmcv
from mmcv import Config

# Weights and Biases
import wandb
print(wandb.__version__)

/home/docsun/anaconda3/envs/mmdetection/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/docsun/anaconda3/envs/mmdetection/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


2.28.2
0.16.3


In [2]:
wandb.login()

wandb: Currently logged in as: juliangeralddcruz. Use `wandb login --relogin` to force relogin


True

In [3]:
@DATASETS.register_module()
class UAVDataset(CustomDataset):

    CLASSES = ('UAV',)

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []
        # convert annotations to middle format
        for image_id in image_list:
            filename = f'{self.img_prefix}/{image_id}.jpg'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}.jpg', width=width, height=height)
    
            # load annotations
            label_prefix = self.img_prefix.replace('images', 'labels')
            lines = mmcv.list_from_file(os.path.join(label_prefix, f'{image_id}.txt'))
    
            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [[float(info) for info in x[4:8]] for x in content]
    
            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []
    
            # filter 'DontCare'
            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_labels.append(cat2label[bbox_name])
                    gt_bboxes.append(bbox)
                else:
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

# 🏈 Model

There are over hundred pre-trained object detectors provided by MMDetection via Model Zoo. Check out the Model Zoo [documentation](https://mmdetection.readthedocs.io/en/v2.21.0/model_zoo.html) page.

You can also customize the model's backbone, neck, head, ROI, and loss. More on customizing the model [here](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_models.html).

### 1️⃣ Download the model

We will be using a pretrained model checkpoint to fine tune on our custom dataset. Let's download the model in the `checkpoints` directory.

In [4]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth \
      -O checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth

mkdir: cannot create directory ‘checkpoints’: File exists
--2024-02-07 18:45:21--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 192.169.122.210, 192.169.122.211, 192.169.122.212, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|192.169.122.210|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



# ⚾️ Configuration

MMDetection relies heavily on a config system. In the cell below, we will be loading a config file and modify few of the methods as per the need of this notebook.

Note that both train and test dataloaders will use the same training samples. This is not a recommended practice but for the sake of a simplified notebook, let's use it. 

Learn more about the MMDetection Config system [here](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html).

### 1️⃣ Load the config file

In [5]:
cfg = Config.fromfile('mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

### 2️⃣ Modify data config

In [6]:
# Define type and path to the images.
cfg.dataset_type = 'UAVDataset'
cfg.data_root = 'data/UAV_DATASET/KITTI_FORMAT'

cfg.data.test.type = 'UAVDataset'
cfg.data.test.data_root = 'data/UAV_DATASET/KITTI_FORMAT'
cfg.data.test.ann_file = 'train.txt'
cfg.data.test.img_prefix = 'test/images'

cfg.data.train.type = 'UAVDataset'
cfg.data.train.data_root = 'data/UAV_DATASET/KITTI_FORMAT'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'train/images'

cfg.data.val.type = 'UAVDataset'
cfg.data.val.data_root = 'data/UAV_DATASET/KITTI_FORMAT'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'val/images'

### 3️⃣ Modify model config

In [7]:
# The number of unique objects in the training data.
cfg.model.roi_head.bbox_head.num_classes = 1
# Use the pretrained model.
cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

### 4️⃣ Modify training config

In [8]:
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Epochs
cfg.runner.max_epochs = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# ⭐️ Set the checkpoint interval.
cfg.checkpoint_config.interval = 1

# Set up working dir to save files and logs.
cfg.work_dir = './mmdetection_logs'
cfg.device = "cuda"

### 5️⃣ Modify evaluation config

In [9]:
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'

# ⭐️ Set the evaluation interval.
cfg.evaluation.interval = 1

# 🎾 Define Weights and Biases Hook

MMDetection comes with a dedicated Weights and Biases Hook - `MMDetWandHook`. MMCV, the parent repository, has a `WandbLoggerHook` that can be used to for basic logging. 

With this dedicated hook, you can:

* log train and eval metrics along with system (CPU/GPU) metrics, 
* visualize the validation dataset as interactive [W&B Tables](https://docs.wandb.ai/guides/data-vis),
* visualize the model prediction as interactive W&B Tables, and
* save the model checkpoints as [W&B Artifacts](https://docs.wandb.ai/guides/artifacts).

To use this hook, you can append a dict to `log_config.hooks`. The `log_config` wraps multiple logger hooks like  the `TextLoggerHook` used below.

There are four important arguments in the `MMDetWandbHook` that can help you get the most out of MMDetection. 

- `init_kwargs`: Use this argument to in-turn pass arguments to `wandb.init`. You can use it to set the W&B project name, set the team name entity if you want to log the runs to a team account, pass the configuration, and more. Check out what all can you pass to `wandb.init` [here](https://docs.wandb.ai/ref/python/init).

- `log_checkpoint`: The model checkpoints are saved at intervals determined by `checkpoint_config.interval` (starred above). If `log_checkpoint` is `True` the saved checkpoints will be saved as versioned W&B Artifact. Note that this feature is dependent on MMCV's [`CheckpointHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.CheckpointHook).

- `log_checkpoint_metadata`: If `log_checkpoint_metadata` is True, every checkpoint artifact will have a metadata associated with it. The metadata contains the evaluation metrics computed on validation data with that checkpoint along with the current epoch. If True, it also marks the checkpoint version with the best evaluation metric with a `best` alias. You can choose the best checkpoint in the W&B Artifacts UI using this.

- `num_eval_images`: At every evaluation interval, the `MMDetWandbHook` logs the model prediction as interactive W&B Tables. The eval interval is determined by `evaluation.interval` (starred above). The number of samples logged is given by `num_eval_images`. The predicted bounding boxes along with the ground truth are logged at every evaluation interval. However, the validation data is logged just once. This Feature is dependent on MMCV's [`EvalHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.EvalHook) or [`DistEvalHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.DistEvalHook).

In [10]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': 'MMDetection-UAV'},
         interval=2,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=6)]

# 🏐 Train

Now that we have the dataset, pretrained model weight, and have defined the configs. Let's stitch them together to train an object detector.

### 1️⃣ Build the Dataset

In [11]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

/home/docsun/Documents/git-repo/Object-Detection-Framework/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


### 2️⃣ Build the Model

In [12]:
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

### 3️⃣ Train with W&B

In [13]:
# Create work_dir
# mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2024-02-07 18:45:24,495 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-02-07 18:45:24,538 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth
2024-02-07 18:45:24,613 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is 

2024-02-07 18:45:30,009 - mmdet - WARNING - No meta information found in the runner. 
wandb: Waiting for artifact val to be committed...

wandb: Committed artifact juliangeralddcruz/MMDetection-UAV/val:v0
wandb:   8 of 8 files downloaded.  
/home/docsun/anaconda3/envs/mmdetection/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2024-02-07 18:45:38,680 - mmdet - INFO - Epoch [1][10/113]	lr: 2.500e-03, eta: 0:07:24, time: 0.330, data_time: 0.210, memory: 2230, loss_rpn_cls: 0.0133, loss_rpn_bbox: 0.0062, loss_cls: 0.2091, acc: 92.6367, loss_bbox: 0.1543, loss: 0.3830
2024-02-07 18:45:39,909 - mmdet - INFO - Epoch [1][20/113]	lr: 2.500e-03, eta: 0:05:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 40/40, 28.1 task/s, elapsed: 1s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-02-07 18:45:54,498 - mmdet - INFO - 
+-------+-----+------+--------+-------+
| class | gts | dets | recall | ap    |
+-------+-----+------+--------+-------+
| UAV   | 40  | 119  | 0.975  | 0.973 |
+-------+-----+------+--------+-------+
| mAP   |     |      |        | 0.973 |
+-------+-----+------+--------+-------+
2024-02-07 18:45:54,535 - mmdet - INFO - Epoch(val) [1][40]	AP50: 0.9730, mAP: 0.9731



---------------iou_thr: 0.5---------------


...

2024-02-07 18:49:33,138 - mmdet - INFO - Epoch [12][10/113]	lr: 2.500e-05, eta: 0:00:14, time: 0.328, data_time: 0.210, memory: 2231, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0029, loss_cls: 0.0249, acc: 98.9258, loss_bbox: 0.0500, loss: 0.0782
2024-02-07 18:49:34,389 - mmdet - INFO - Epoch [12][20/113]	lr: 2.500e-05, eta: 0:00:13, time: 0.125, data_time: 0.007, memory: 2231, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0025, loss_cls: 0.0253, acc: 98.9355, loss_bbox: 0.0489, loss: 0.0770
2024-02-07 18:49:35,653 - mmdet - INFO - Epoch [12][30/113]	lr: 2.500e-05, eta: 0:00:11, time: 0.126, data_time: 0.008, memory: 2231, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0018, loss_cls: 0.0219, acc: 99.1309, loss_bbox: 0.0466, loss: 0.0704
2024-02-07 18:49:36,981 - mmdet - INFO - Epoch [12][40/113]	lr: 2.500e-05, eta: 0:00:10, time: 0.133, data_time: 0.009, memory: 2231, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0024, loss_cls: 0.0256, acc: 98.9258, loss_bbox: 0.0617, loss: 0.0898
2024-02-07 18:49:38,283 - mm

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 40/40, 28.0 task/s, elapsed: 1s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-02-07 18:49:48,838 - mmdet - INFO - 
+-------+-----+------+--------+-------+
| class | gts | dets | recall | ap    |
+-------+-----+------+--------+-------+
| UAV   | 40  | 49   | 0.975  | 0.969 |
+-------+-----+------+--------+-------+
| mAP   |     |      |        | 0.969 |
+-------+-----+------+--------+-------+
2024-02-07 18:49:48,874 - mmdet - INFO - Epoch(val) [12][40]	AP50: 0.9690, mAP: 0.9688



---------------iou_thr: 0.5---------------


learning_rate,███████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▄▂▄▄▅▅▆▄▇▆▇▆█▇█▅▇█▇██▆██▅▇██▆▇▇▇▇▇█▇▇▇▇
train/loss,█▆▇▆▅▅▄▄▄▃▃▃▃▂▂▂▄▃▂▃▃▂▃▁▂▅▂▁▂▃▃▁▂▂▂▁▂▂▂▂
train/loss_bbox,█▆▆▆▄▅▃▄▄▃▂▃▄▂▂▂▄▃▂▄▃▂▂▂▂▆▃▂▂▃▃▂▂▂▂▁▂▂▂▂
train/loss_cls,▇▅█▅█▅▅▃▅▂▄▃▃▁▂▂▅▂▁▂▁▂▃▁▁▅▂▁▁▄▃▁▃▂▂▁▃▂▁▂
train/loss_rpn_bbox,▃▅█▃▃█▄▄▃▂▄▃▄▂▃▃▃▃▂▃▂▁▄▂▂▇▃▂▇▂▃▂▃▂▂▂▁▂▂▂
train/loss_rpn_cls,▆▁▇▁█▂▆▁▁▂▁▁▁▂▁▃▁▁▁▁▁▁▆▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂
val/AP50,██▄▅██▅▁▂▁▂▂
val/mAP,██▄▅█▇▅▁▂▁▂▂
learning_rate,3e-05


### 4️⃣ Notes on using `MMDetWandbHook`. 

Using `MMDetWandbHook` is easy and in most cases it will throw friendly `UserWarning` if something is not quite right. However in the best interest, here are some of things and best practices you should keep in mind:

* The `MMDetWandbHook` depends on `CheckpointHook` for logging the checkpoints as W&B Artifacts and `EvalHook`/`DistEvalHook` for logging validation data and model predictions. If anyone or both aren't available, this hook will give `UserWarning` and not cause any error. 

* The priority of both `CheckpointHook` and `EvalHook`/`DistEvalHook` should be more than `MMDetWandbHook`. 

* The validation data is logged once as `val_data` W&B Table. The evaluation tables, use reference to this data thus you will not be uploading the same data multiple times. 

* If you want to log the configuration to W&B, pass this key-value pair `'config': cfg._cfg_dict.to_dict()` to `init_kwargs`. 